# Revisiting code
## To do
* Avoid crosswalk: write a function that alters organizations: gets rid of words like agency, extra white space, characters such as /, semicolons, dba, change to lower_case
* See if that helps.

In [1]:
import numpy as np
import pandas as pd
import math
from siuba import *
from calitp import *
import intake

import shared_utils
from plotnine import *
import altair as alt
import altair_saver
from shared_utils import altair_utils 
import cpi

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
#NB ONLY
import data_prep_refactored
import data_prep 
import agency_crosswalk
pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = 250

In [3]:
#Load in data
df_5311 = data_prep_refactored.load_grantprojects()
vehicles = data_prep_refactored.load_cleaned_vehiclesdata()
organizations = data_prep_refactored.load_cleaned_organizations_data()
airtable = data_prep_refactored.load_airtable()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [4]:
#Agencies that ONLY appear in Black Cat  for function merged_dataframe()
BC_Agency_List = ['County of Los Angeles - Department of Public Works',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Department of Transportation',
 'Glenn County Transportation Commission',
 'Stanislaus County Public Works - Transit Division',
 'Alpine County Community Development',
 'Fresno Council of Governments',
 'Greyhound Lines, Inc.']

In [5]:
crosswalk = {'City of Chowchilla ': 'City of Chowchilla, dba: Chowchilla Area Transit ',
     'City of Dinuba ':  'City of Dinuba',
     'Modoc Transportation Agency': 'Modoc Transportation Agency',
     'Butte County Association of Governments/ Butte Regional Transit': 'Butte County Association of Governments',
     'Calaveras County Public Works':  'Calaveras Transit Agency',
     'City of Escalon ':  'City of Escalon, dba: eTrans',
     'County of Mariposa':  'Mariposa County Transit, dba: Mari-Go',
     'County of Shasta Department of Public Works':  'County of Shasta Department of Public Works',
     'County of Siskiyou': 'County of Siskiyou, dba: Siskiyou County Transit',
     'County of Tulare': 'Tulare County Area Transit',
     'Eureka Transit Service':  'City of Eureka, dba: Eureka Transit Service',
     'Kern Regional Transit':  'Kern Regional Transit',
     'Livermore Amador Valley Transit Authority':  'Livermore / Amador Valley Transit Authority',
     'Placer County Public Works (TART & PCT)': 'County of Placer, dba: Placer County Department of Public Works',
     'Plumas County Transportation Commission': 'Plumas County Transportation Commission',
     'San Luis Obispo Regional Transit Authority':  'San Luis Obispo Regional Transit Authority',
     'Sonoma County Transit':  'County of Sonoma, dba: Sonoma County Transit',
     'Sunline Transit Agency':  'SunLine Transit Agency',
     'Tehama County Transit Agency': 'Tehama County',
     'Trinity County Department of Transportation ':  'Trinity County',
     'Tuolumne County Transit Agency (TCTA)':  'Tuolumne County Transit',
     'Amador Transit':  'Amador Regional Transit System',
     'City of Corcoran - Corcoran Area Transit':  'City of Corcoran, dba: Corcoran Area Transit',
     'Yosemite Area Regional Transportation System ':  'Yosemite Area Regional Transportation System',
     'County Connection (Central Contra Costa Transit Authority)': 'Central Contra Costa Transit Authority, dba: COUNTY CONNECTION',
     'Calaveras Transit Agency ': 'Calaveras Transit Agency'}

In [6]:
# Inflation table
def inflation_table(base_year):
    cpi.update()
    series_df = cpi.series.get(area="U.S. city average").to_dataframe()
    inflation_df = (series_df[series_df.year >= 2008]
           .pivot_table(index='year', values='value', aggfunc='mean')
           .reset_index()
          )
    denominator = inflation_df.value.loc[inflation_df.year==base_year].iloc[0]

    inflation_df = inflation_df.assign(
        inflation = inflation_df.value.divide(denominator)
    )
    
    return inflation_df

def adjust_prices(df):
    
    cols =  ["allocationamount",
             "encumbered_amount",
             "expendedamount",
             "activebalance",
             "closedoutbalance"]
    
    ##get cpi table 
    cpi = inflation_table(2021)
    cpi.update
    cpi = (cpi>>select(_.year, _.value))
    cpi_dict = dict(zip(cpi['year'], cpi['value']))
    
    
    for col in cols:
        multiplier = df["project_year"].map(cpi_dict)  
    
        ##using 270.97 for 2021 dollars
        df[f"adjusted_{col}"] = ((df[col] * 270.97) / multiplier)
    return df


## Function 1: Merge NTD Vehicles with  Airtable

In [7]:
'''
Merge NTD Vehicles data set with GTFS info from Airtable
'''
def ntd_airtable_merge():
    #Importing the dataframes: organizations is Cal ITP's Airtable
    #Vehicles is from NTD's vechicles dataset
    organizations = data_prep_refactored.load_cleaned_organizations_data() #Delete data_prep_refactored once moving to the scripts
    vehicles = data_prep_refactored.load_cleaned_vehiclesdata()
    #merge the 2 datasets on the left, since there are many more entries on the left
    m1 = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'])
    return m1

In [8]:
ntd_gtfs = ntd_airtable_merge()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

## Function 2: Merge M1 with 5311 Info Black Cat

In [11]:
'''
Merge m1 with 5311 info from BlackCat
'''
def ntd_airtable_5311_merge():
    ntd = data_prep_refactored.load_cleaned_vehiclesdata()
    df_5311 = data_prep_refactored.load_grantprojects()
    #FIRST MERGE: 
    #5311 info from Black Cat on the left.
    m2 = (pd.merge(df_5311, ntd_gtfs,  how='left', left_on=['organization_name'], 
                      right_on=['name'], indicator=True)
            )
    #Subset out a df with left only matches
    Left_only = m2[(m2._merge.str.contains("left_only", case= False))] 
    
    #Take organizations left and make it into a list
    Left_orgs = Left_only['organization_name'].drop_duplicates().tolist()
    
    #Delete  left only matches from original df 
    m2 = m2[~m2.organization_name.isin(Left_orgs)]
    
    #SECOND MERGE:
    #As we deleted organizations that were "left only",
    #make a data frame with these values, replace their names with ones in NTD data
    fail = df_5311[df_5311.organization_name.isin(Left_orgs)]
    
    #replacing organization names from Black Cat with agency names from m1 
    #REDO 
    fail['organization_name'].replace(crosswalk, inplace= True)
    
    #Merging the "failed" dataframe with m1 (NTD and GTFS Airtable info) 
    m3 = pd.merge(fail, ntd_gtfs,  how='left', left_on=['organization_name'], right_on=['agency'])
    
    #Concat: 
    m4 = pd.concat([m2, m3])
    return m4
    

In [12]:
ntd_gtfs_bc = ntd_airtable_5311_merge()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

## Functions 3 & 4: Metrics

In [13]:
def fleet_size_rating(df): 
    #FLEET SIZE 
    #First grabbing only one row for each agency into a new data frame 
    Fleet_size = df.groupby(['organization_name',]).agg({'total_vehicles':'max'}).reindex()
    #Get percentiles in objects for total vehicle.
    p75 = df.total_vehicles.quantile(0.75).astype(float)
    p25 = df.total_vehicles.quantile(0.25).astype(float)
    p50 = df.total_vehicles.quantile(0.50).astype(float)
    #Function for fleet size
    def fleet_size (row):
        if ((row.total_vehicles > 0) and (row.total_vehicles < p25)):
            return "Small"
        elif ((row.total_vehicles > p25) and (row.total_vehicles < p75)):
            return "Medium"
        elif ((row.total_vehicles > p50) and (row.total_vehicles > p75 )):
               return "Large"
        else:
            return "No Info"
    df["fleet_size"] = df.apply(lambda x: fleet_size(x), axis=1)
  
    return df

In [14]:
#Flag BlackCat only values
def blackcat_only(row):
    #If there are no values for GTFS, reporter type, and fleet size, then we can safely say
    #This organization is not registered by Cal ITP or NTD
    if ((row.GTFS == 'None') and (row.reporter_type == 'None') and row.fleet_size == 'No Info'):
        return "1"
    else:
        return "0"

## Function 5: Clean DF 
* Cleaning up merged df & adding metric flags 

In [15]:
def final_df():
    df1 = ntd_airtable_5311_merge()
    airtable2 = data_prep_refactored.load_airtable()
    #Manually replace Klamath
    #Klamath does not appear in NTD data so we missed it when we merged NTD & Cal ITP on NTD ID
    df1.loc[(df1['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "itp_id"] = "436"
    df1.loc[(df1['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "gtfs_schedule_status"] = "needed"
    
    #Change ITP ID ID to be floats & 0 so parquet will work
    df1['itp_id'] = df1['itp_id'].fillna(0)
    df1.loc[(df1['itp_id'] == '436'), "itp_id"] = 436
    
    #Del & rename columns
    df2 = df1.drop(columns=['name','gtfs_schedule_status','agency'])
    
    #Call inflation function to add $ columns with adjusted values for inflation 
    df2 = adjust_prices(df1)
    
    #Apply fleet size() function
    df2 = fleet_size_rating(df1)
    
    #Merge df1 with the new airtable stuff 
    final = pd.merge(df1, airtable2, how='left', left_on='name', right_on='name')
    
    #Concatenate the two GTFS cols together into one column 
    final["GTFS"] = final["gtfs_static_status"] + '_' + final["gtfs_realtime_status"]
     
    #Drop old columns
    final = final.drop(columns = ['gtfs_static_status','gtfs_realtime_status','_merge'])
    
    #Fill NA by data types 
    final.fillna(final.dtypes.replace({'float64': 0.0, 'object': 'None'}), inplace=True)
    
    #Apply Black Cat only function 
    final["Is_Agency_In_BC_Only_1_means_Yes"] = final.apply(lambda x: blackcat_only(x), axis=1)
    return final


In [18]:
ntd_gtfs_bc = final_df()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

## Function 6: Aggregate data

In [21]:
'''
Aggregate: sums up original df to one row for each organization with aggregated statistics. 
GTFS, district, planning authority, and mppo rtpa info also included
'''
def aggregated_df(): 
    #Read original df & airtable
    df = final_df()
    
    #Aggregate all vehicles 16+ above into one col 
    column_names = ['_16_20', '_21_25', '_26_30', '_31_60']
    df['_16plus']= df[column_names].sum(axis=1)
    
    #duplicate allocationamount col so we can get the mean an organiztaion received.
    df['allocation_mean'] = df['allocationamount']
    
    #grab the max of all vehicle information & sum of all monetary cols from Black Cat
    df = (df.groupby(['organization_name',
                     'reporter_type', 
                     'fleet_size', 
                     'ntd_id', 
                     'itp_id','mpo_rtpa','GTFS','caltrans_district','planning_authority'])
          .agg({'Is_Agency_In_BC_Only_1_means_Yes':'max',
                'allocation_mean': 'mean',
                'allocationamount':'sum',
                'encumbered_amount':'sum',
                                         'expendedamount':'sum',
                                         'activebalance':'sum',
                                         'closedoutbalance':'sum',
                                         'adjusted_allocationamount':'sum',
                                         'adjusted_expendedamount':'sum',
                                         'adjusted_encumbered_amount':'sum',
                                         'adjusted_activebalance':'sum',
                                         'adjusted_closedoutbalance':'sum',                                                
                                         'total_vehicles':'max',
                                         'average_age_of_fleet__in_years_':'max',
                                         'average_lifetime_miles_per_vehicle':'max',
                                         'Automobiles':'max',
                                         'Bus':'max',
                                         'Other':'max',
                                         'Train':'max',
                                         'Van':'max',
                                         'automobiles_door':'max',
                                         'bus_doors':'max',
                                         'van_doors':'max',
                                         'train_doors':'max',
                                         'doors_sum':'max',
                                         '_0_9':'max',
                                         '_10_12':'max',
                                         '_13_15':'max',
                                         '_16_20':'max',
                                         '_21_25':'max',
                                         '_26_30':'max',
                                         '_31_60':'max',
                                         '_16plus':'max',
                                         '_60plus':'max'}).reset_index()
         ) 
   
    return df 
    

In [22]:
agg_df_for_analysis = aggregated_df()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [23]:
agg_df_for_analysis.shape

(87, 43)

In [24]:
agg_df_for_analysis[['organization_name','Is_Agency_In_BC_Only_1_means_Yes','reporter_type', 'fleet_size','GTFS']]

,organization_name,Is_Agency_In_BC_Only_1_means_Yes,reporter_type,fleet_size,GTFS
0,Alpine County Community Development,1,None,No Info,None
1,Amador Regional Transit System,0,Rural Reporter,Medium,Static OK_RT Incomplete
2,Butte County Association of Governments,0,Full Reporter,Medium,Static OK_RT Incomplete
3,Calaveras Transit Agency,0,Rural Reporter,Medium,None
4,"Central Contra Costa Transit Authority, dba: C...",0,Full Reporter,Large,None
5,City of Arcata,0,Rural Reporter,No Info,Static OK_RT OK
6,City of Arvin,0,Rural Reporter,No Info,Static OK_RT Incomplete
7,City of Auburn,0,Rural Reporter,Small,Static OK_RT Incomplete
8,City of California City,0,Rural Reporter,Small,Static OK_RT Incomplete
9,"City of Chowchilla, dba: Chowchilla Area Transit",0,Rural Reporter,Small,Static OK_RT Incomplete


## Double Check

In [25]:
original = data_prep.aggregated_merged_df()

/home/jovyan/data-analyses/5311/data_prep.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [26]:
original.shape

(87, 44)

In [27]:

set(original.columns).difference(set(agg_df_for_analysis.columns))

{'GTFS_Status',
 'allocationamount_mean',
 'gtfs_realtime_status',
 'gtfs_static_status'}

In [28]:
original.head(2)

,organization_name,allocationamount_mean,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,reporter_type,fleet_size,ntd_id,itp_id,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status,GTFS_Status
0,Alpine County Community Development,48766.57,341366.00,293366.00,159901.31,181464.69,0,363613.49,173611.38,315613.45,190002.10,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,None,No Info,NaN,0,NaN,NaN,NaN,NaN,NaN,None
1,Amador Regional Transit System,189817.38,1518539.00,1279114.00,1067984.00,450555.00,0,1612119.29,1151645.28,1372694.08,460474.01,0.00,24.00,4.07,0.00,0.00,0.00,0.00,0.00,21.00,0.00,0.00,21.00,0.00,21.00,18.00,3.00,1.00,1.00,1.00,0.00,0.00,2.00,0.00,Rural Reporter,Medium,9R02-91000,11.00,10 - Stockton,NaN,Amador County Transportation Commission,Static OK,RT Incomplete,Only Static


In [29]:
agg_df_for_analysis.head(2)

,organization_name,reporter_type,fleet_size,ntd_id,itp_id,mpo_rtpa,GTFS,caltrans_district,planning_authority,Is_Agency_In_BC_Only_1_means_Yes,allocation_mean,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus
0,Alpine County Community Development,None,No Info,None,0.00,None,None,None,None,1,48766.57,341366.00,293366.00,159901.31,181464.69,0,363613.49,173611.38,315613.45,190002.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Amador Regional Transit System,Rural Reporter,Medium,9R02-91000,11.00,None,Static OK_RT Incomplete,10 - Stockton,Amador County Transportation Commission,0,189817.38,1518539.00,1279114.00,1067984.00,450555.00,0,1612119.29,1151645.28,1372694.08,460474.01,0.00,24.00,4.07,0.00,0.00,0.00,0.00,0.00,21.00,0.00,0.00,21.00,0.00,21.00,18.00,3.00,1.00,1.00,1.00,0.00,0.00,2.00,0.00


In [30]:
org_original = set(original.organization_name.unique().tolist())
org_script = set(agg_df_for_analysis.organization_name.unique().tolist())

In [31]:
org_original-org_script

set()

In [32]:
org_script-org_original

set()

In [33]:
agg_df_for_analysis['GTFS'].value_counts()

Static OK_RT Incomplete            53
Static OK_RT OK                    15
None                               12
Static Incomplete_RT Incomplete     5
Static Incomplete_RT OK             2
Name: GTFS, dtype: int64

In [34]:
agg_df_for_analysis['Is_Agency_In_BC_Only_1_means_Yes'].value_counts()

0    78
1     9
Name: Is_Agency_In_BC_Only_1_means_Yes, dtype: int64

In [35]:
agg_df_for_analysis.groupby(['caltrans_district', 'organization_name','GTFS']).agg({'allocation_mean':'max'})

allocation_mean
caltrans_district    organization_name                                  GTFS                                            
01 - Eureka          City of Arcata                                     Static OK_RT OK                         82111.43
                     City of Eureka, dba: Eureka Transit Service        Static OK_RT OK                        173923.00
                     Humboldt Transit Authority                         Static OK_RT OK                        272277.52
                     Lake Transit Authority                             Static OK_RT OK                        280841.11
                     Mendocino Transit Authority                        Static OK_RT Incomplete                382165.09
                     Redwood Coast Transit Authority                    Static OK_RT Incomplete                135621.50
02 - Redding         County of Siskiyou, dba: Siskiyou County Transit   Static OK_RT Incomplete                256852.83
                     Lassen Transit Service Agency                      Static OK_RT Incomplete                199618.50
                     Modoc Transportation Agency                        Static OK_RT Incomplete                 47578.14
                     Trinity County                                     Static OK_RT Incomplete                130729.18
03 - Marysville      Butte County Association of Governments            Static OK_RT Incomplete                490548.92
                     City of Auburn                                     Static OK_RT Incomplete                 69865.83
                     Colusa County Transit Agency                       Static OK_RT Incomplete                105024.29
                     County of Placer, dba: Placer County Department... Static OK_RT Incomplete                348464.43
                     El Dorado County Transit Authority                 Static OK_RT Incomplete                508602.67
                     Town of Truckee                                    Static OK_RT Incomplete                 92407.50
                     Yolo County Transportation District                Static OK_RT Incomplete                140163.17
                     Yuba-Sutter Transit Authority                      Static OK_RT Incomplete                 89890.21
04 - Oakland         City of Dixon                                      Static OK_RT Incomplete                170903.56
                     City of Fairfield                                  Static OK_RT OK                        100000.00
                     City of Rio Vista                                  Static OK_RT Incomplete                 83945.56
                     County of Sonoma, dba: Sonoma County Transit       Static OK_RT OK                         95388.94
                     Livermore / Amador Valley Transit Authority        Static Incomplete_RT Incomplete         43871.50
                     Marin County Transit District                      Static Incomplete_RT OK                214857.67
                     Napa Valley Transportation Authority               Static OK_RT Incomplete                242859.82
                     San Mateo County Transit District                  Static Incomplete_RT Incomplete        159212.67
                     Santa Clara Valley Transportation Authority        Static OK_RT OK                         81896.17
05 - San Luis Obispo City of Guadalupe                                  Static OK_RT Incomplete                 62625.67
                     City of Santa Maria                                Static OK_RT OK                        112253.00
                     City of Solvang                                    Static OK_RT Incomplete                201995.50
                     Monterey-Salinas Transit                           Static Incomplete_RT OK                299488.25
                     San Benito County Local Transportation Authority   Static OK_RT Incomplete                3

In [36]:
agg_df_for_analysis['organization_name'].nunique()

87

In [37]:
def pick_column_and_aggregate(df1, df2, col):
    if df1[col].sum() == df2[col].sum():
        print("PASS")
    else:
        print(f"{col}: FAIL")

In [38]:
my_columns = ['allocationamount',
       'encumbered_amount', 'expendedamount', 'activebalance',
       'closedoutbalance', 'adjusted_allocationamount',
       'adjusted_expendedamount', 'adjusted_encumbered_amount',
       'adjusted_activebalance', 'adjusted_closedoutbalance', 'total_vehicles',]

In [39]:
for c in my_columns:
    pick_column_and_aggregate(agg_df_for_analysis, original, c)

PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS
